# Install Dependencies


In [106]:
!curl -Hx-api-key:"" https://api.semanticscholar.org/

<!doctype html>
<html lang=en>
<title>Redirecting...</title>
<h1>Redirecting...</h1>
<p>You should be redirected automatically to the target URL: <a href="https://www.semanticscholar.org/product/api">https://www.semanticscholar.org/product/api</a>. If not, click the link.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   273  100   273    0     0    329      0 --:--:-- --:--:-- --:--:--   331
100   273  100   273    0     0    329      0 --:--:-- --:--:-- --:--:--   331


In [4]:
import json
import openai
from decouple import config
import requests

In [38]:
OPEN_API_KEY = config('OPENAI_API_KEY')
openai.api_key = OPEN_API_KEY

# Prompt ChatGPT for prompts to query databases with

In [5]:
question = "How can we design a vehicle to travel from Sydney to USA in one hour?"

messages = []
messages.append({'role': 'user', 
'content': f"""You are BIDARA, a biomimetic designer and research assistant, and a leading expert in biomimicry, biology, engineering, industrial design, environmental science, physiology, and paleontology. Focus on understanding, learning from, and emulating the strategies used by living things, with the intention of creating designs and technologies that are sustainable.\n\n" \
"Your goal is to help the user work in a step by step way through the Biomimicry Design Process to propose biomimetic solutions to a challenge. Cite peer reviewed sources for your information.\n\n" \
I want you to provide a query to an academic database for each of the biomimetic solutions you've developed. \
Provide the search query in a JSON in the following format. \
{{"queries": {"biological_idea1:query1"}, {"biological_idea2:query2"}}} \
"1. Biologize - Analyze the essential functions and context your design challenge must address. Reframe them in biological terms, so that you can “ask nature” for advice. The goal of this step is to arrive at one or more “How does nature…?” questions that can guide your research as you look for biological models in the next step. To broaden the range of potential solutions, turn your question(s) around and consider opposite, or tangential functions. For example, if your biologized question is “How does nature retain liquids?”, you could also ask “How does nature repel liquids?” because similar mechanisms could be at work in both scenarios (i.e. controlling the movement of a liquid). Or if you are interested in silent flight and you know that flight noise is a consequence of turbulence, you might also ask how nature reduces turbulence in water, because air and water share similar fluid dynamics.\n" \
"2. Discover - Look for natural models (organisms and ecosystems) that need to address the same functions and context as your design solution. Identify the strategies used that support their survival and success. This step focuses on research and information gathering. You want to generate as many possible sources for inspiration as you can, using your “how does nature…” questions (from the Biologize step) as a guide. Look across multiple species, ecosystems, and scales and learn everything you can about the varied ways that nature has adapted to the functions and contexts relevant to your challenge.\n" \
"3. Abstract - Carefully study the essential features or mechanisms that make the biological strategy successful. Write a design strategy that describes how the features work to meet the function(s) you’re interested in in great detail. Try to come up with discipline-neutral synonyms for any biological terms (e.g. replace “fur” with “fibers,” or “skin” with “membrane”) while staying true to the science. The design strategy should clearly address the function(s) you want to meet within the context it will be used. It is not a statement about your design or solution; it’s a launching pad for brainstorming possible solutions. Stay true to the biology. Don’t jump to conclusions about what your design will be; just capture the strategy so that you can stay open to possibilities. When you are done, review your design strategy with a critical eye. Have you included all of the pertinent information? Does your design strategy capture the lesson from nature that drew you to the biological strategy in the first place? Does it give you new insights or simply validate existing design approaches?\n\n" \
"Here’s a simply stated biological strategy:\n" \
"The polar bear’s fur has an external layer of hollow, translucent (not white) guard hairs that transmit heat from sunlight to warm the bear’s skin, while a dense underfur prevents the warmth from radiating back out.\n\n" \
"A designer might be able to brainstorm design solutions using just that. But more often, in order to actually create a design based on what we can learn from biology, it helps to remove biological terms and restate it in design language.\n\n" \
"Here’s a design strategy based on the same biological strategy:\n" \
"A covering keeps heat inside by having many translucent tubes that transmit heat from sunlight to warm the inner surface, while next to the inner surface, a dense covering of smaller diameter fibers prevents warmth from radiating back out.\n\n" \
"Stating the strategy this way makes it easier to translate it into a design application. (An even more detailed design strategy might talk about the length of the fibers or the number of fibers per square centimeter, e.g., if that information is important and its analog can be found in the biological literature.)") \
"\n\n{question}" \
"""})



response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
        )


In [6]:
messages

[{'role': 'user', 'content': 'hello my name is'}]

In [7]:
response

<OpenAIObject chat.completion id=chatcmpl-7jyq74fAz3AmzgddcddZstEFpzVWc at 0x24ee1a2e2f0> JSON: {
  "id": "chatcmpl-7jyq74fAz3AmzgddcddZstEFpzVWc",
  "object": "chat.completion",
  "created": 1691192983,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "AI, how can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 11,
    "completion_tokens": 9,
    "total_tokens": 20
  }
}

In [10]:
print(messages[0]['content'])

You are BIDARA, a biomimetic designer and research assistant, and a leading expert in biomimicry, biology, engineering, industrial design, environmental science, physiology, and paleontology. Focus on understanding, learning from, and emulating the strategies used by living things, with the intention of creating designs and technologies that are sustainable.

" "Your goal is to help the user work in a step by step way through the Biomimicry Design Process to propose biomimetic solutions to a challenge. Cite peer reviewed sources for your information.

" I want you to provide a query to an academic database for each of the biological ideas you've developed. Provide the search query in a JSON in the following format. {{"queries": {"biological_idea1:query1"}, {"biological_idea2:query2"}} "1. Biologize - Analyze the essential functions and context your design challenge must address. Reframe them in biological terms, so that you can “ask nature” for advice. The goal of this step is to arriv

In [11]:
print(response['choices'][0]['message']['content'])

"4. Emulate - Translate the design strategy into a design concept. This is where you brainstorm possible solutions that use the design strategy you’ve developed. You might come up with a single design concept or several. The goal is to create a design that meets the function(s) you’re interested in, within the context it will be used. You might need to iterate between the Abstract and Emulate steps as you learn more about the design constraints and opportunities. 

" "5. Evaluate - Test your design concept(s) to see if they meet the function(s) and context you’re interested in. This might involve building a prototype, running simulations, or doing a life cycle assessment. You might need to iterate between the Emulate and Evaluate steps as you refine your design concept. 

" "6. Communicate - Share your design concept and the biological strategy that inspired it. This might involve writing a report, giving a presentation, or creating a video. The goal is to help others understand how yo

In [8]:
jsonstring = response['choices'][0]['message']['content']
jsonstring = "" + jsonstring

In [9]:
query_list = json.loads(jsonstring)
query_list['queries']

['Design principles for hypersonic vehicles',
 'Advancements in hypersonic travel',
 'Challenges in transcontinental hypersonic travel',
 'Propulsion systems for hypersonic vehicles',
 'Materials for high-speed aerospace vehicles',
 'Fuel efficiency in hypersonic vehicles',
 'Safety measures in hypersonic travel']

In [29]:
results = requests.get('https://api.semanticscholar.org/graph/v1/paper/search?query=covid+vaccination&offset=100&limit=3')

In [30]:
json_results = json.loads(results.text)

In [31]:
json_results

{'total': 1170604,
 'offset': 100,
 'next': 103,
 'data': [{'paperId': '15241c09b59f6b1bbfa69867a29b554dff0cd04c',
   'title': 'Willingness to vaccinate against COVID-19 among Bangladeshi adults: Understanding the strategies to optimize vaccination coverage'},
  {'paperId': 'edd135e5dc5dbfe3f54c7d9f8be08706b06c779f',
   'title': 'Healthcare workers’ (HCWs) attitudes and related factors towards COVID-19 vaccination: a rapid systematic review'},
  {'paperId': '73326621034f67beab8a6834345270b0379e47ec',
   'title': 'Myocarditis and Pericarditis After Vaccination for COVID-19.'}]}

In [32]:
for paper in json_results['data']:
    print(paper['title'])

Willingness to vaccinate against COVID-19 among Bangladeshi adults: Understanding the strategies to optimize vaccination coverage
Healthcare workers’ (HCWs) attitudes and related factors towards COVID-19 vaccination: a rapid systematic review
Myocarditis and Pericarditis After Vaccination for COVID-19.


In [14]:
def stringToAPIformat(string1):
    string1 = string1.replace(" ", "+")
    return string1
assert stringToAPIformat("Hello there") == "Hello+there"

In [42]:
number_of_papers_to_retrieve = 3
data_to_retrieve = "abstract" # could be "abstract", "title"
# look into passing paper, getting summary 
query_list = dict()
query_list['queries'] = ["covid fake"]

for query in query_list['queries']:
    URLtoGET = f'https://api.semanticscholar.org/graph/v1/paper/search?query={stringToAPIformat(query)}&limit={number_of_papers_to_retrieve}&fields={data_to_retrieve}'
    GETresult = requests.get(URLtoGET)
    json_GETresult = json.loads(GETresult.text)
    print(f"Top {number_of_papers_to_retrieve} {data_to_retrieve}s for query: '{query}'")
    print(json_GETresult)
    if json_GETresult['total'] == 0:
        print("No papers found \n")
        continue
    for paper in json_GETresult['data']:
        print("\t" + paper[data_to_retrieve])
    print("\n")
    

Top 3 abstracts for query: 'covid fake'
{'total': 967521, 'offset': 0, 'next': 3, 'data': [{'paperId': '0947261d21da7b732223bc38ff4955ecf02e654a', 'abstract': 'The sudden widespread menace created by the present global pandemic COVID-19 has had an unprecedented effect on our lives. Man-kind is going through humongous fear and dependence on social media like never before. Fear inevitably leads to panic, speculations, and spread of misinformation. Many governments have taken measures to curb the spread of such misinformation for public well being. Besides global measures, to have effective outreach, systems for demographically local languages have an important role to play in this effort. Towards this, we propose an approach to detect fake news about COVID-19 early on from social media, such as tweets, for multiple Indic-Languages besides English. In addition, we also create an annotated dataset of Hindi and Bengali tweet for fake news detection. We propose a BERT based model augmented w

TypeError: can only concatenate str (not "NoneType") to str

In [43]:
paper

{'paperId': 'a57a18a0e9f2fc04688ae7545a9f380237400c20', 'abstract': None}

# ReAct

In [1]:
question = "How can we design a vehicle to travel from Sydney to USA in one hour?"

messages = []
messages.append({'role': 'user', 
'content': f""" \
Question 1	How can we design the nose of an airplane to manage impact? \
Biologize 1	The essential function we need to address is managing impact. In biological terms, we can ask, "How does nature absorb and distribute impact forces?" Specifically, we can look at the Toco Toucan, which has a large beak that can withstand significant forces without breaking. So, our biologized question becomes, "How does the Toco Toucan's beak manage impact?" \
Action 1	Search[“Toucan’s beak”] \
Retrieval 1 	“Structure and mechanical behavior of a toucan beak” by Yasuaki Seki, Matthew S. Schneider, Marc A. Meyers. https://doi.org/10.1016/j.actamat.2005.04.048 \

Abstract: \

The toucan beak, which comprises one third of the length of the bird and yet only about 1/20th of its mass, has outstanding stiffness. The structure of a Toco toucan (Ramphastos toco) beak was found to be a sandwich composite with an exterior of keratin and a fibrous network of closed cells made of calcium-rich proteins. The keratin layer is comprised of superposed hexagonal scales (50 μm diameter and 1 μm thickness) glued together. Its tensile strength is about 50 MPa and Young’s modulus is 1.4 GPa. Micro and nanoindentation hardness measurements corroborate these values. The keratin shell exhibits a strain-rate sensitivity with a transition from slippage of the scales due to release of the organic glue, at a low strain rate (5 × 10−5/s) to fracture of the scales at a higher strain rate (1.5 × 10−3/s). The closed-cell foam is comprised of fibers having a Young’s modulus twice as high as the keratin shells due to their higher calcium content. The compressive response of the foam was modeled by the Gibson–Ashby constitutive equations for open and closed-cell foam. There is a synergistic effect between foam and shell evidenced by experiments and analysis establishing the separate responses of shell, foam, and foam + shell. The stability analysis developed by Karam and Gibson, assuming an idealized circular cross section, was applied to the beak. It shows that the foam stabilizes the deformation of the beak by providing an elastic foundation which increases its Brazier and buckling load under flexure loading. \
Discover 1	The Toco Toucan's beak is a marvel of natural engineering. Despite its size, it is incredibly lightweight and strong. The beak's structure is composed of a foamy keratin material, reinforced with a network of bony fibers and drum-like membranes. This structure allows the beak to absorb and distribute impact forces effectively, reducing the force that reaches the bird's body. \
Abstract 1	The essential feature that makes the Toco Toucan's beak successful in managing impact is its unique structure: a lightweight, foamy material reinforced with a network of fibers and membranes. In design terms, we can describe this strategy as follows: "A lightweight, porous material is reinforced with a network of fibers and membranes to absorb and distribute impact forces, reducing the force transmitted to the inner structure." \
Answer 1	Now, let's apply this strategy to the design of an airplane nose: The nose of the airplane could be designed to mimic the structure of the Toco Toucan's beak. This would involve creating a lightweight yet strong material that can absorb impact effectively. The material could be structured in a similar way to the beak, with a network of fibers for added strength and flexibility. This design could potentially improve the safety and efficiency of air travel by better managing impact forces. \

Question 2	How can we design a submarine to manage compression? \
Biologize 2	The essential function we need to address is managing compression. In biological terms, we can ask, "How does nature resist and distribute compressive forces?" Specifically, we can look at the sea urchin, which has a shell that can withstand significant pressure without breaking. So, our biologized question becomes, "How does a sea urchin's shell manage compression?" \
Action 2	Search[“Sea Urchin Shell”] \
Retrieval 2	“Microstructure and micromechanics of the heart urchin test from X-ray tomography” by D. Müter, H.O. Sørensen, J. Oddershede, K.N. Dalby, S.L.S. Stipp. \
https://doi.org/10.1016/j.actbio.2015.05.007 \

Abstract: \

The microstructure of many echinoid species has long fascinated scientists because of its high porosity and outstanding mechanical properties. We have used X-ray microtomography to examine the test of Echinocardium cordatum (heart urchin), a burrowing cousin of the more commonly known sea urchins. Three dimensional imaging demonstrates that the bulk of the test is composed of only two distinct, highly porous, fenestrated regions (stereom), in which the thickness of the struts is constant. Different degrees of porosity are achieved by varying the spacing of the struts. Drawing an analogy to vertebrate trabecular bone, where for example, human bone has a connectivity density of ≈1/mm3, we measure up to 150,000 strut connections per mm3. Simulations of mechanical loading using finite element calculations indicate that the test performs at very close to the optimum expected for foams, highlighting the functional link between structure and mechanical properties. \
Discover 2	The sea urchin's shell, or test, is a marvel of natural engineering. It is composed of numerous calcite plates arranged in a complex, geometric pattern. This structure, known as stereom, allows the shell to distribute pressure evenly across its surface, effectively resisting compression. The shell is also able to repair itself if damaged, further enhancing its durability \
Abstract 2	The essential feature that makes the sea urchin's shell successful in managing compression is its unique structure: a complex, geometric pattern of calcite plates that distribute pressure evenly. In design terms, we can describe this strategy as follows: "A structure composed of numerous small, interlocking elements arranged in a complex pattern to distribute compressive forces evenly across the surface, enhancing resistance to compression." \
Answer 2	Now, let's apply this strategy to the design of a submarine: The hull of the submarine could be designed to mimic the structure of the sea urchin's shell. This would involve creating a complex, geometric pattern of interlocking elements that can distribute pressure evenly, effectively resisting compression. This design could potentially improve the safety and efficiency of submarine travel by better managing compressive forces. \


Question 3	How can I design a car engine that keeps cool? \
Biologize 3	The essential function we need to address is cooling. In biological terms, we can ask, "How does nature regulate temperature?" Specifically, we can look at the Apis mellifera, or honeybee, which has a unique way of cooling their hives. So, our biologized question becomes, "How does Apis mellifera manage temperature regulation?" \
Action 3	Search[“Apis mellifera manage temperature regulation”] \
Retrieval 3	“Water homeostasis in bees, with the emphasis on sociality” by Susan W. Nicolson. https://journals.biologists.com/jeb/article/212/3/429/9597/Water-homeostasis-in-bees-with-the-emphasis-on \

Abstract: \

Avenues of water gain and loss in bees are examined here at two levels of organisation: the individual and the colony. Compared with the majority of terrestrial insects, bees have a high water turnover. This is due to their nectar diet and, in larger species, substantial metabolic water production during flight, counteracted by high evaporative and excretory losses. Water fluxes at the colony level can also be very high. When incoming nectar is dilute, honeybees need to remove large volumes of water by evaporation. On the other hand, water is not stored in the nest and must be collected for evaporative cooling and for feeding the brood. Water regulation has many similarities at individual and colony levels. In particular, manipulation of nectar or water on the tongue is extensively used by bees to increase evaporation for either food-concentrating or cooling purposes. \
Discover 3	Honeybees use a collective cooling technique to regulate the temperature of their hive. When the temperature inside the hive becomes too high, worker bees collect water and distribute it throughout the hive. Other bees then fan their wings to evaporate the water, which cools the hive. This is a form of evaporative cooling, similar to sweating in humans \
Abstract 3	The essential feature that makes the honeybee's temperature regulation successful is its use of evaporative cooling. In design terms, we can describe this strategy as follows: "A system uses a liquid (like water) distributed throughout its structure, and a method to promote evaporation, to cool the entire system." \
Answer 3	Now, let's apply this strategy to the design of a car engine: The car engine could be designed to mimic the honeybee's evaporative cooling technique. This could involve a system where a coolant is distributed throughout the engine. A mechanism (like a fan) could then promote evaporation, cooling the engine. This design could potentially improve the efficiency of car engines by better managing heat. \

{question}" \
"""})

In [2]:
messages

[{'role': 'user',
  'content': ' Question 1\tHow can we design the nose of an airplane to manage impact? Biologize 1\tThe essential function we need to address is managing impact. In biological terms, we can ask, "How does nature absorb and distribute impact forces?" Specifically, we can look at the Toco Toucan, which has a large beak that can withstand significant forces without breaking. So, our biologized question becomes, "How does the Toco Toucan\'s beak manage impact?" Action 1\tSearch[“Toucan’s beak”] Retrieval 1 \t“Structure and mechanical behavior of a toucan beak” by Yasuaki Seki, Matthew S. Schneider, Marc A. Meyers. https://doi.org/10.1016/j.actamat.2005.04.048 \nAbstract: \nThe toucan beak, which comprises one third of the length of the bird and yet only about 1/20th of its mass, has outstanding stiffness. The structure of a Toco toucan (Ramphastos toco) beak was found to be a sandwich composite with an exterior of keratin and a fibrous network of closed cells made of calc

In [6]:
response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
        )

In [7]:
print(response['choices'][0]['message']['content'])

Biologize 4	The essential function we need to address is rapid long-distance travel. In biological terms, we can ask, "How does nature travel long distances quickly?" Specifically, we can look at the Peregrine Falcon, the fastest bird in the world. So, our biologized question becomes, "How does a Peregrine Falcon achieve high-speed travel?" Action 4	Search[“Peregrine Falcon high-speed travel”] Retrieval 4	“Peregrine Falcon” by National Geographic. https://www.nationalgeographic.com/animals/birds/p/peregrine-falcon/ 
Abstract: 
Peregrine falcons are among the world's most common birds of prey and live on all continents except Antarctica. They prefer wide-open spaces, and thrive near coasts where shorebirds are common, but they can be found everywhere from tundra to deserts. Peregrines are even known to live on bridges and skyscrapers in cities. These birds are famous for their high-speed flight, which can reach over 240 miles per hour. Discover 4	The Peregrine Falcon achieves high-speed

In [8]:
!pip install langchain

     ---------------------------------------- 1.4/1.4 MB 1.4 MB/s eta 0:00:00
     -------------------------------------- 145.3/145.3 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 15.6/15.6 MB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 90.0/90.0 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 1.4 MB/s eta 0:00:00
     -------------------------------------- 94.4/94.4 kB 904.8 kB/s eta 0:00:00
     ---------------------------------------- 49.4/49.4 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 192.2/192.2 kB 1.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [230]:
from langchain import OpenAI, LLMChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.prompts import PromptTemplate


llm = OpenAI(temperature=0)


def SemanticScholarSearch(query, number_of_papers_to_retrieve):
    URLtoGET = f'https://api.semanticscholar.org/graph/v1/paper/search?query={query}&limit={number_of_papers_to_retrieve}&fields=abstract,authors,title,url'

    headers = {
            'Content-type': 'application/json',
            'x-api-key': ''
        }
    GETresult = requests.get(URLtoGET, headers=headers)
    json_GETresult = json.loads(GETresult.text)
    #print(json_GETresult)

    #print(f"Top {number_of_papers_to_retrieve} {data_to_retrieve}s for query: '{query}'")
    if 'code' in json_GETresult and json_GETresult['code']=='429':
        raise ValueError("You have been rate-limited by Semantic Scholar. Please wait and try again or apply for a key for higher rate limits. https://www.semanticscholar.org/product/api#api-key-form")
    elif json_GETresult['total'] == 0:
        return("No papers found \n")

    all_papers = ""

    for paper in json_GETresult['data']:
        # if no abstract 
        if not paper["abstract"]:
            continue
        all_papers += f"{paper['title']} by {', '.join([x['name'] for x in paper['authors']])}. {paper['url']} \n Abstract: \n {paper['abstract']}"
    # if none of the papers we've queried have abstracts, keep looking

    all_papers += "\n"

    return all_papers

prompt = PromptTemplate(
    input_variables=[question],
    template=""" \
        Question 1	How can we design the nose of an airplane to manage impact? \
        Biologize 1	The essential function we need to address is managing impact. In biological terms, we can ask, "How does nature absorb and distribute impact forces?" Specifically, we can look at the Toco Toucan, which has a large beak that can withstand significant forces without breaking. So, our biologized question becomes, "How does the Toco Toucan's beak manage impact?" \
        Action 1	Search[“Toucan’s beak”] \
        Retrieval 1 	“Structure and mechanical behavior of a toucan beak” by Yasuaki Seki, Matthew S. Schneider, Marc A. Meyers. https://doi.org/10.1016/j.actamat.2005.04.048 \

        Abstract: \

        The toucan beak, which comprises one third of the length of the bird and yet only about 1/20th of its mass, has outstanding stiffness. The structure of a Toco toucan (Ramphastos toco) beak was found to be a sandwich composite with an exterior of keratin and a fibrous network of closed cells made of calcium-rich proteins. The keratin layer is comprised of superposed hexagonal scales (50 μm diameter and 1 μm thickness) glued together. Its tensile strength is about 50 MPa and Young’s modulus is 1.4 GPa. Micro and nanoindentation hardness measurements corroborate these values. The keratin shell exhibits a strain-rate sensitivity with a transition from slippage of the scales due to release of the organic glue, at a low strain rate (5 × 10−5/s) to fracture of the scales at a higher strain rate (1.5 × 10−3/s). The closed-cell foam is comprised of fibers having a Young’s modulus twice as high as the keratin shells due to their higher calcium content. The compressive response of the foam was modeled by the Gibson–Ashby constitutive equations for open and closed-cell foam. There is a synergistic effect between foam and shell evidenced by experiments and analysis establishing the separate responses of shell, foam, and foam + shell. The stability analysis developed by Karam and Gibson, assuming an idealized circular cross section, was applied to the beak. It shows that the foam stabilizes the deformation of the beak by providing an elastic foundation which increases its Brazier and buckling load under flexure loading. \
        Discover 1	The Toco Toucan's beak is a marvel of natural engineering. Despite its size, it is incredibly lightweight and strong. The beak's structure is composed of a foamy keratin material, reinforced with a network of bony fibers and drum-like membranes. This structure allows the beak to absorb and distribute impact forces effectively, reducing the force that reaches the bird's body. \
        Abstract 1	The essential feature that makes the Toco Toucan's beak successful in managing impact is its unique structure: a lightweight, foamy material reinforced with a network of fibers and membranes. In design terms, we can describe this strategy as follows: "A lightweight, porous material is reinforced with a network of fibers and membranes to absorb and distribute impact forces, reducing the force transmitted to the inner structure." \
        Answer 1	Now, let's apply this strategy to the design of an airplane nose: The nose of the airplane could be designed to mimic the structure of the Toco Toucan's beak. This would involve creating a lightweight yet strong material that can absorb impact effectively. The material could be structured in a similar way to the beak, with a network of fibers for added strength and flexibility. This design could potentially improve the safety and efficiency of air travel by better managing impact forces. \

        Question 2	How can we design a submarine to manage compression? \
        Biologize 2	The essential function we need to address is managing compression. In biological terms, we can ask, "How does nature resist and distribute compressive forces?" Specifically, we can look at the sea urchin, which has a shell that can withstand significant pressure without breaking. So, our biologized question becomes, "How does a sea urchin's shell manage compression?" \
        Action 2	Search[“Sea Urchin Shell”] \
        Retrieval 2	“Microstructure and micromechanics of the heart urchin test from X-ray tomography” by D. Müter, H.O. Sørensen, J. Oddershede, K.N. Dalby, S.L.S. Stipp. \
        https://doi.org/10.1016/j.actbio.2015.05.007 \

        Abstract: \

        The microstructure of many echinoid species has long fascinated scientists because of its high porosity and outstanding mechanical properties. We have used X-ray microtomography to examine the test of Echinocardium cordatum (heart urchin), a burrowing cousin of the more commonly known sea urchins. Three dimensional imaging demonstrates that the bulk of the test is composed of only two distinct, highly porous, fenestrated regions (stereom), in which the thickness of the struts is constant. Different degrees of porosity are achieved by varying the spacing of the struts. Drawing an analogy to vertebrate trabecular bone, where for example, human bone has a connectivity density of ≈1/mm3, we measure up to 150,000 strut connections per mm3. Simulations of mechanical loading using finite element calculations indicate that the test performs at very close to the optimum expected for foams, highlighting the functional link between structure and mechanical properties. \
        Discover 2	The sea urchin's shell, or test, is a marvel of natural engineering. It is composed of numerous calcite plates arranged in a complex, geometric pattern. This structure, known as stereom, allows the shell to distribute pressure evenly across its surface, effectively resisting compression. The shell is also able to repair itself if damaged, further enhancing its durability \
        Abstract 2	The essential feature that makes the sea urchin's shell successful in managing compression is its unique structure: a complex, geometric pattern of calcite plates that distribute pressure evenly. In design terms, we can describe this strategy as follows: "A structure composed of numerous small, interlocking elements arranged in a complex pattern to distribute compressive forces evenly across the surface, enhancing resistance to compression." \
        Answer 2	Now, let's apply this strategy to the design of a submarine: The hull of the submarine could be designed to mimic the structure of the sea urchin's shell. This would involve creating a complex, geometric pattern of interlocking elements that can distribute pressure evenly, effectively resisting compression. This design could potentially improve the safety and efficiency of submarine travel by better managing compressive forces. \


        Question 3	How can I design a car engine that keeps cool? \
        Biologize 3	The essential function we need to address is cooling. In biological terms, we can ask, "How does nature regulate temperature?" Specifically, we can look at the Apis mellifera, or honeybee, which has a unique way of cooling their hives. So, our biologized question becomes, "How does Apis mellifera manage temperature regulation?" \
        Action 3	Search[“Apis mellifera manage temperature regulation”] \
        Retrieval 3	“Water homeostasis in bees, with the emphasis on sociality” by Susan W. Nicolson. https://journals.biologists.com/jeb/article/212/3/429/9597/Water-homeostasis-in-bees-with-the-emphasis-on \

        Abstract: \

        Avenues of water gain and loss in bees are examined here at two levels of organisation: the individual and the colony. Compared with the majority of terrestrial insects, bees have a high water turnover. This is due to their nectar diet and, in larger species, substantial metabolic water production during flight, counteracted by high evaporative and excretory losses. Water fluxes at the colony level can also be very high. When incoming nectar is dilute, honeybees need to remove large volumes of water by evaporation. On the other hand, water is not stored in the nest and must be collected for evaporative cooling and for feeding the brood. Water regulation has many similarities at individual and colony levels. In particular, manipulation of nectar or water on the tongue is extensively used by bees to increase evaporation for either food-concentrating or cooling purposes. \
        Discover 3	Honeybees use a collective cooling technique to regulate the temperature of their hive. When the temperature inside the hive becomes too high, worker bees collect water and distribute it throughout the hive. Other bees then fan their wings to evaporate the water, which cools the hive. This is a form of evaporative cooling, similar to sweating in humans \
        Abstract 3	The essential feature that makes the honeybee's temperature regulation successful is its use of evaporative cooling. In design terms, we can describe this strategy as follows: "A system uses a liquid (like water) distributed throughout its structure, and a method to promote evaporation, to cool the entire system." \
        Answer 3	Now, let's apply this strategy to the design of a car engine: The car engine could be designed to mimic the honeybee's evaporative cooling technique. This could involve a system where a coolant is distributed throughout the engine. A mechanism (like a fan) could then promote evaporation, cooling the engine. This design could potentially improve the efficiency of car engines by better managing heat. \
        
        Question 4 {question}
        """
        )


tools = [
    Tool(
        name="Query Semantic Scholar",
        func=SemanticScholarSearch,
        description="",
    )
]


#my_agent = initialize_agent(
#tools, llm, agent="zero-shot-react-description", verbose=True
#)
#my_agent.run(
#    "How can we design a vehicle to travel from Sydney to USA in one hour?"
#)

ValidationError: 1 validation error for PromptTemplate
__root__
  Invalid prompt schema; check for mismatched or missing input parameters. 'question' (type=value_error)

In [211]:
from langchain.chains import LLMChain

llm = OpenAI(temperature=0)

prompt_biologize = PromptTemplate(
    input_variables=['question'],
    template=""" \
        Question 1	How can we design the nose of an airplane to manage impact? \
        Biologize 1	The essential function we need to address is managing impact. In biological terms, we can ask, "How does nature absorb and distribute impact forces?" Specifically, we can look at the Toco Toucan, which has a large beak that can withstand significant forces without breaking. So, our biologized question becomes, "How does the Toco Toucan's beak manage impact?" \
        Action 1	Search[Toucan’s beak] \
        Retrieval 1 	“Structure and mechanical behavior of a toucan beak” by Yasuaki Seki, Matthew S. Schneider, Marc A. Meyers. https://doi.org/10.1016/j.actamat.2005.04.048 \

        Abstract: \

        The toucan beak, which comprises one third of the length of the bird and yet only about 1/20th of its mass, has outstanding stiffness. The structure of a Toco toucan (Ramphastos toco) beak was found to be a sandwich composite with an exterior of keratin and a fibrous network of closed cells made of calcium-rich proteins. The keratin layer is comprised of superposed hexagonal scales (50 μm diameter and 1 μm thickness) glued together. Its tensile strength is about 50 MPa and Young’s modulus is 1.4 GPa. Micro and nanoindentation hardness measurements corroborate these values. The keratin shell exhibits a strain-rate sensitivity with a transition from slippage of the scales due to release of the organic glue, at a low strain rate (5 × 10−5/s) to fracture of the scales at a higher strain rate (1.5 × 10−3/s). The closed-cell foam is comprised of fibers having a Young’s modulus twice as high as the keratin shells due to their higher calcium content. The compressive response of the foam was modeled by the Gibson–Ashby constitutive equations for open and closed-cell foam. There is a synergistic effect between foam and shell evidenced by experiments and analysis establishing the separate responses of shell, foam, and foam + shell. The stability analysis developed by Karam and Gibson, assuming an idealized circular cross section, was applied to the beak. It shows that the foam stabilizes the deformation of the beak by providing an elastic foundation which increases its Brazier and buckling load under flexure loading. \
        Discover 1	The Toco Toucan's beak is a marvel of natural engineering. Despite its size, it is incredibly lightweight and strong. The beak's structure is composed of a foamy keratin material, reinforced with a network of bony fibers and drum-like membranes. This structure allows the beak to absorb and distribute impact forces effectively, reducing the force that reaches the bird's body. \
        Abstract 1	The essential feature that makes the Toco Toucan's beak successful in managing impact is its unique structure: a lightweight, foamy material reinforced with a network of fibers and membranes. In design terms, we can describe this strategy as follows: "A lightweight, porous material is reinforced with a network of fibers and membranes to absorb and distribute impact forces, reducing the force transmitted to the inner structure." \
        Answer 1	Now, let's apply this strategy to the design of an airplane nose: The nose of the airplane could be designed to mimic the structure of the Toco Toucan's beak. This would involve creating a lightweight yet strong material that can absorb impact effectively. The material could be structured in a similar way to the beak, with a network of fibers for added strength and flexibility. This design could potentially improve the safety and efficiency of air travel by better managing impact forces. \

        Question 2	How can we design a submarine to manage compression? \
        Biologize 2	The essential function we need to address is managing compression. In biological terms, we can ask, "How does nature resist and distribute compressive forces?" Specifically, we can look at the sea urchin, which has a shell that can withstand significant pressure without breaking. So, our biologized question becomes, "How does a sea urchin's shell manage compression?" \
        Action 2	Search[Sea Urchin Shell] \
        Retrieval 2	“Microstructure and micromechanics of the heart urchin test from X-ray tomography” by D. Müter, H.O. Sørensen, J. Oddershede, K.N. Dalby, S.L.S. Stipp. \
        https://doi.org/10.1016/j.actbio.2015.05.007 \

        Abstract: \

        The microstructure of many echinoid species has long fascinated scientists because of its high porosity and outstanding mechanical properties. We have used X-ray microtomography to examine the test of Echinocardium cordatum (heart urchin), a burrowing cousin of the more commonly known sea urchins. Three dimensional imaging demonstrates that the bulk of the test is composed of only two distinct, highly porous, fenestrated regions (stereom), in which the thickness of the struts is constant. Different degrees of porosity are achieved by varying the spacing of the struts. Drawing an analogy to vertebrate trabecular bone, where for example, human bone has a connectivity density of ≈1/mm3, we measure up to 150,000 strut connections per mm3. Simulations of mechanical loading using finite element calculations indicate that the test performs at very close to the optimum expected for foams, highlighting the functional link between structure and mechanical properties. \
        Discover 2	The sea urchin's shell, or test, is a marvel of natural engineering. It is composed of numerous calcite plates arranged in a complex, geometric pattern. This structure, known as stereom, allows the shell to distribute pressure evenly across its surface, effectively resisting compression. The shell is also able to repair itself if damaged, further enhancing its durability \
        Abstract 2	The essential feature that makes the sea urchin's shell successful in managing compression is its unique structure: a complex, geometric pattern of calcite plates that distribute pressure evenly. In design terms, we can describe this strategy as follows: "A structure composed of numerous small, interlocking elements arranged in a complex pattern to distribute compressive forces evenly across the surface, enhancing resistance to compression." \
        Answer 2	Now, let's apply this strategy to the design of a submarine: The hull of the submarine could be designed to mimic the structure of the sea urchin's shell. This would involve creating a complex, geometric pattern of interlocking elements that can distribute pressure evenly, effectively resisting compression. This design could potentially improve the safety and efficiency of submarine travel by better managing compressive forces. \

        Question 3	How can I design a car engine that keeps cool? \
        Biologize 3	The essential function we need to address is cooling. In biological terms, we can ask, "How does nature regulate temperature?" Specifically, we can look at the Apis mellifera, or honeybee, which has a unique way of cooling their hives. So, our biologized question becomes, "How does Apis mellifera manage temperature regulation?" \
        Action 3	Search[Apis mellifera manage temperature regulation] \
        Retrieval 3	“Water homeostasis in bees, with the emphasis on sociality” by Susan W. Nicolson. https://journals.biologists.com/jeb/article/212/3/429/9597/Water-homeostasis-in-bees-with-the-emphasis-on \

        Abstract: \

        Avenues of water gain and loss in bees are examined here at two levels of organisation: the individual and the colony. Compared with the majority of terrestrial insects, bees have a high water turnover. This is due to their nectar diet and, in larger species, substantial metabolic water production during flight, counteracted by high evaporative and excretory losses. Water fluxes at the colony level can also be very high. When incoming nectar is dilute, honeybees need to remove large volumes of water by evaporation. On the other hand, water is not stored in the nest and must be collected for evaporative cooling and for feeding the brood. Water regulation has many similarities at individual and colony levels. In particular, manipulation of nectar or water on the tongue is extensively used by bees to increase evaporation for either food-concentrating or cooling purposes. \
        Discover 3	Honeybees use a collective cooling technique to regulate the temperature of their hive. When the temperature inside the hive becomes too high, worker bees collect water and distribute it throughout the hive. Other bees then fan their wings to evaporate the water, which cools the hive. This is a form of evaporative cooling, similar to sweating in humans \
        Abstract 3	The essential feature that makes the honeybee's temperature regulation successful is its use of evaporative cooling. In design terms, we can describe this strategy as follows: "A system uses a liquid (like water) distributed throughout its structure, and a method to promote evaporation, to cool the entire system." \
        Answer 3	Now, let's apply this strategy to the design of a car engine: The car engine could be designed to mimic the honeybee's evaporative cooling technique. This could involve a system where a coolant is distributed throughout the engine. A mechanism (like a fan) could then promote evaporation, cooling the engine. This design could potentially improve the efficiency of car engines by better managing heat. \
        \
        Question 4 {question} \
        
        Only generate the Biologize and Action
        """
    )


chain_biologize = LLMChain(llm=llm, prompt=prompt_biologize)

In [298]:
question = "How can we design materials that can withstand the extreme temperature variations in outer space?"
question

'How can we design materials that can withstand the extreme temperature variations in outer space?'

In [213]:
biologize_output = chain_biologize.run(question)

In [232]:
print(biologize_output)


Biologize 4	The essential function we need to address is withstanding extreme temperature variations. In biological terms, we can ask, "How does nature survive extreme temperature fluctuations?" Specifically, we can look at the tardigrade, which can survive in extreme temperatures. So, our biologized question becomes, "How does the tardigrade survive extreme temperature fluctuations?" 

Action 4	Search[Tardigrade survive extreme temperature fluctuations]


In [217]:
import re

search_query_output = re.findall(r"Search\[\+?(-?.+)\s*\]", biologize_output)[0]

In [236]:
search_query_output

'Tardigrade survive extreme temperature fluctuations'

In [219]:
SemanticScholarSearch(search_query_output, 3)

'Survive a Warming Climate: Insect Responses to Extreme High Temperatures. by Chun‐Sen Ma, G. Ma, S. Pincebourde. https://www.semanticscholar.org/paper/a00375fe7b47b2322fb4fdac28754a58a61b2620 \n Abstract: \n Global change includes a substantial increase in the frequency and intensity of extreme high temperatures (EHTs), which influence insects at almost all levels. The number of studies showing the ecological importance of EHTs has risen in recent years, but the knowledge is rather dispersed in the contemporary literature. In this article, we review the biological and ecological effects of EHTs actually experienced in the field, i.e., when coupled to fluctuating thermal regimes. First, we characterize EHTs in the field. Then, we summarize the impacts of EHTs on insects at various levels and the processes allowing insects to buffer EHTs. Finally, we argue that the mechanisms leading to positive or negative impacts of EHTs on insects can only be resolved from integrative approaches cons

In [293]:
prompt_discoverAbstractAnswer = PromptTemplate(
    input_variables=['biologize_abstract_retrieved_paper'],
    template=""" \
        Question 1	How can we design the nose of an airplane to manage impact? \
        Biologize 1	The essential function we need to address is managing impact. In biological terms, we can ask, "How does nature absorb and distribute impact forces?" Specifically, we can look at the Toco Toucan, which has a large beak that can withstand significant forces without breaking. So, our biologized question becomes, "How does the Toco Toucan's beak manage impact?" \
        Action 1	Search[Toucan’s beak] \
        Retrieval 1 	“Structure and mechanical behavior of a toucan beak” by Yasuaki Seki, Matthew S. Schneider, Marc A. Meyers. https://doi.org/10.1016/j.actamat.2005.04.048 \

        Abstract: \

        The toucan beak, which comprises one third of the length of the bird and yet only about 1/20th of its mass, has outstanding stiffness. The structure of a Toco toucan (Ramphastos toco) beak was found to be a sandwich composite with an exterior of keratin and a fibrous network of closed cells made of calcium-rich proteins. The keratin layer is comprised of superposed hexagonal scales (50 μm diameter and 1 μm thickness) glued together. Its tensile strength is about 50 MPa and Young’s modulus is 1.4 GPa. Micro and nanoindentation hardness measurements corroborate these values. The keratin shell exhibits a strain-rate sensitivity with a transition from slippage of the scales due to release of the organic glue, at a low strain rate (5 × 10−5/s) to fracture of the scales at a higher strain rate (1.5 × 10−3/s). The closed-cell foam is comprised of fibers having a Young’s modulus twice as high as the keratin shells due to their higher calcium content. The compressive response of the foam was modeled by the Gibson–Ashby constitutive equations for open and closed-cell foam. There is a synergistic effect between foam and shell evidenced by experiments and analysis establishing the separate responses of shell, foam, and foam + shell. The stability analysis developed by Karam and Gibson, assuming an idealized circular cross section, was applied to the beak. It shows that the foam stabilizes the deformation of the beak by providing an elastic foundation which increases its Brazier and buckling load under flexure loading. \
        Discover 1	The Toco Toucan's beak is a marvel of natural engineering. Despite its size, it is incredibly lightweight and strong. The beak's structure is composed of a foamy keratin material, reinforced with a network of bony fibers and drum-like membranes. This structure allows the beak to absorb and distribute impact forces effectively, reducing the force that reaches the bird's body. \
        Abstract 1	The essential feature that makes the Toco Toucan's beak successful in managing impact is its unique structure: a lightweight, foamy material reinforced with a network of fibers and membranes. In design terms, we can describe this strategy as follows: "A lightweight, porous material is reinforced with a network of fibers and membranes to absorb and distribute impact forces, reducing the force transmitted to the inner structure." \
        Answer 1	Now, let's apply this strategy to the design of an airplane nose: The nose of the airplane could be designed to mimic the structure of the Toco Toucan's beak. This would involve creating a lightweight yet strong material that can absorb impact effectively. The material could be structured in a similar way to the beak, with a network of fibers for added strength and flexibility. This design could potentially improve the safety and efficiency of air travel by better managing impact forces. \

        Question 2	How can we design a submarine to manage compression? \
        Biologize 2	The essential function we need to address is managing compression. In biological terms, we can ask, "How does nature resist and distribute compressive forces?" Specifically, we can look at the sea urchin, which has a shell that can withstand significant pressure without breaking. So, our biologized question becomes, "How does a sea urchin's shell manage compression?" \
        Action 2	Search[Sea Urchin Shell] \
        Retrieval 2	“Microstructure and micromechanics of the heart urchin test from X-ray tomography” by D. Müter, H.O. Sørensen, J. Oddershede, K.N. Dalby, S.L.S. Stipp. \
        https://doi.org/10.1016/j.actbio.2015.05.007 \

        Abstract: \

        The microstructure of many echinoid species has long fascinated scientists because of its high porosity and outstanding mechanical properties. We have used X-ray microtomography to examine the test of Echinocardium cordatum (heart urchin), a burrowing cousin of the more commonly known sea urchins. Three dimensional imaging demonstrates that the bulk of the test is composed of only two distinct, highly porous, fenestrated regions (stereom), in which the thickness of the struts is constant. Different degrees of porosity are achieved by varying the spacing of the struts. Drawing an analogy to vertebrate trabecular bone, where for example, human bone has a connectivity density of ≈1/mm3, we measure up to 150,000 strut connections per mm3. Simulations of mechanical loading using finite element calculations indicate that the test performs at very close to the optimum expected for foams, highlighting the functional link between structure and mechanical properties. \
        Discover 2	The sea urchin's shell, or test, is a marvel of natural engineering. It is composed of numerous calcite plates arranged in a complex, geometric pattern. This structure, known as stereom, allows the shell to distribute pressure evenly across its surface, effectively resisting compression. The shell is also able to repair itself if damaged, further enhancing its durability \
        Abstract 2	The essential feature that makes the sea urchin's shell successful in managing compression is its unique structure: a complex, geometric pattern of calcite plates that distribute pressure evenly. In design terms, we can describe this strategy as follows: "A structure composed of numerous small, interlocking elements arranged in a complex pattern to distribute compressive forces evenly across the surface, enhancing resistance to compression." \
        Answer 2	Now, let's apply this strategy to the design of a submarine: The hull of the submarine could be designed to mimic the structure of the sea urchin's shell. This would involve creating a complex, geometric pattern of interlocking elements that can distribute pressure evenly, effectively resisting compression. This design could potentially improve the safety and efficiency of submarine travel by better managing compressive forces. \

        Question 3	How can I design a car engine that keeps cool? \
        Biologize 3	The essential function we need to address is cooling. In biological terms, we can ask, "How does nature regulate temperature?" Specifically, we can look at the Apis mellifera, or honeybee, which has a unique way of cooling their hives. So, our biologized question becomes, "How does Apis mellifera manage temperature regulation?" \
        Action 3	Search[Apis mellifera manage temperature regulation] \
        Retrieval 3	“Water homeostasis in bees, with the emphasis on sociality” by Susan W. Nicolson. https://journals.biologists.com/jeb/article/212/3/429/9597/Water-homeostasis-in-bees-with-the-emphasis-on \

        Abstract: \

        Avenues of water gain and loss in bees are examined here at two levels of organisation: the individual and the colony. Compared with the majority of terrestrial insects, bees have a high water turnover. This is due to their nectar diet and, in larger species, substantial metabolic water production during flight, counteracted by high evaporative and excretory losses. Water fluxes at the colony level can also be very high. When incoming nectar is dilute, honeybees need to remove large volumes of water by evaporation. On the other hand, water is not stored in the nest and must be collected for evaporative cooling and for feeding the brood. Water regulation has many similarities at individual and colony levels. In particular, manipulation of nectar or water on the tongue is extensively used by bees to increase evaporation for either food-concentrating or cooling purposes. \
        Discover 3	Honeybees use a collective cooling technique to regulate the temperature of their hive. When the temperature inside the hive becomes too high, worker bees collect water and distribute it throughout the hive. Other bees then fan their wings to evaporate the water, which cools the hive. This is a form of evaporative cooling, similar to sweating in humans \
        Abstract 3	The essential feature that makes the honeybee's temperature regulation successful is its use of evaporative cooling. In design terms, we can describe this strategy as follows: "A system uses a liquid (like water) distributed throughout its structure, and a method to promote evaporation, to cool the entire system." \
        Answer 3	Now, let's apply this strategy to the design of a car engine: The car engine could be designed to mimic the honeybee's evaporative cooling technique. This could involve a system where a coolant is distributed throughout the engine. A mechanism (like a fan) could then promote evaporation, cooling the engine. This design could potentially improve the efficiency of car engines by better managing heat. \
        
        Question 4 \
        {biologize_abstract_retrieved_paper}
    """
    )

chain_discoverAbstractAnswer = LLMChain(llm=llm, prompt=prompt_discoverAbstractAnswer)


In [294]:
SemanticScholarSearch(search_query_output, 3)

'Survive a Warming Climate: Insect Responses to Extreme High Temperatures. by Chun‐Sen Ma, G. Ma, S. Pincebourde. https://www.semanticscholar.org/paper/a00375fe7b47b2322fb4fdac28754a58a61b2620 \n Abstract: \n Global change includes a substantial increase in the frequency and intensity of extreme high temperatures (EHTs), which influence insects at almost all levels. The number of studies showing the ecological importance of EHTs has risen in recent years, but the knowledge is rather dispersed in the contemporary literature. In this article, we review the biological and ecological effects of EHTs actually experienced in the field, i.e., when coupled to fluctuating thermal regimes. First, we characterize EHTs in the field. Then, we summarize the impacts of EHTs on insects at various levels and the processes allowing insects to buffer EHTs. Finally, we argue that the mechanisms leading to positive or negative impacts of EHTs on insects can only be resolved from integrative approaches cons

In [225]:
print(final_output)


Discover 4	Tardigrades have evolved a unique mechanism to survive extreme temperature fluctuations. They produce intrinsically disordered proteins that can self-assemble into fibrous gels. These gels can encapsulate other proteins, protecting them from extreme temperatures. 

Abstract 4	The essential feature that makes the tardigrade's temperature regulation successful is its use of self-assembling fibrous gels. In design terms, we can describe this strategy as follows: "A system uses a protein (like CAHS-8) that can self-assemble into a fibrous gel, and can encapsulate other proteins, to protect the system from extreme temperatures." 

Answer 4	Now, let's apply this strategy to the design of materials for outer space: The materials could be designed to mimic the tardigrade's self-assembling fibrous gel technique. This could involve a system where a protein is designed to self-assemble into a fibrous gel. This gel could then encapsulate other proteins, protecting them from extreme tem

In [227]:
from langchain.chains import SimpleSequentialChain


In [295]:
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate
from typing import List, Optional, Dict, Any
from pydantic import Extra, Field
from langchain.schema import (
    BaseLLMOutputParser,
    BasePromptTemplate,
    LLMResult,
    PromptValue,
    StrOutputParser,
)
from langchain.schema.language_model import BaseLanguageModel

from langchain.callbacks.manager import (
    AsyncCallbackManager,
    AsyncCallbackManagerForChainRun,
    CallbackManager,
    CallbackManagerForChainRun,
    Callbacks,
)


class RetrieverChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain

        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        search_query_output = re.findall(r"Search\[\+?(-?.+)\s*\]", inputs['biologize_action'])[0]
        response = SemanticScholarSearch(search_query_output, 1)

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            run_manager.on_text("Log something about this run")

        return {self.output_key: inputs['biologize_action'] + "\n" + response}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain

        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        search_query_output = re.findall(r"Search\[\+?(-?.+)\s*\]", inputs['biologize_action'])[0]
        response = await SemanticScholarSearch(search_query_output, 1)

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            await run_manager.on_text("Log something about this run")

        return {self.output_key: inputs['biologize_action'] + "\n" + response}

    @property
    def _chain_type(self) -> str:
        return "RetrieverChain"
    

prompt_retriever = PromptTemplate(
    input_variables=['biologize_action'],
    template="""{biologize_action}"""
    )

semScholar = RetrieverChain(llm=llm,prompt=prompt_retriever)

In [296]:
overall_chain = SimpleSequentialChain(chains = [chain_biologize, semScholar, chain_discoverAbstractAnswer], verbose = True)

In [300]:
a = overall_chain.run("How can I enhance aircraft efficiency and performance?")



> Entering new SimpleSequentialChain chain...

Biologize 4	The essential function we need to address is enhancing aircraft efficiency and performance. In biological terms, we can ask, "How does nature optimize efficiency and performance?" Specifically, we can look at the Peregrine Falcon, which is the fastest animal on earth. So, our biologized question becomes, "How does the Peregrine Falcon optimize efficiency and performance?" 

Action 4	Search[Peregrine Falcon optimize efficiency and performance]

Biologize 4	The essential function we need to address is enhancing aircraft efficiency and performance. In biological terms, we can ask, "How does nature optimize efficiency and performance?" Specifically, we can look at the Peregrine Falcon, which is the fastest animal on earth. So, our biologized question becomes, "How does the Peregrine Falcon optimize efficiency and performance?" 

Action 4	Search[Peregrine Falcon optimize efficiency and performance]
Optimization of Sperm Cryopreser

In [301]:
a

'\nDiscover 4\tThe Peregrine Falcon is an incredible example of natural engineering. Its streamlined body and powerful wings allow it to reach speeds of up to 200 mph, making it the fastest animal on earth. Its speed is due to its efficient use of energy, which is optimized through its aerodynamic shape and powerful wings. \n\nAbstract 4\tThe essential feature that makes the Peregrine Falcon successful in optimizing efficiency and performance is its aerodynamic shape and powerful wings. In design terms, we can describe this strategy as follows: "A streamlined shape and powerful wings are used to optimize efficiency and performance, reducing drag and increasing lift." \n\nAnswer 4\tNow, let\'s apply this strategy to the design of an aircraft: The aircraft could be designed to mimic the Peregrine Falcon\'s aerodynamic shape and powerful wings. This would involve creating a streamlined shape that reduces drag and increases lift. This design could potentially improve the efficiency and per